In [119]:
'''basic packages'''
import os
import pandas as pd
import numpy as np
from sklearn import datasets, tree, linear_model
import sklearn.ensemble as ek
import seaborn as sns
import queue
import cv2          #影像處理
import scipy.misc   #影像處理
from tqdm import tqdm_notebook as tqdm #進度條
import matplotlib.pyplot as plt
plt.rcParams['font.family']='SimHei' #顯示中文
import matplotlib.lines as mlines
%matplotlib inline
import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [120]:
# Load in the train datasets
train = pd.read_csv('/data/examples/may_the_4_be_with_u/show_me_the_money/train.csv',
                    names=['id','step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','Fraud'],
                    encoding = "utf-8", header=None)
train = train.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest', 'Fraud':'isFraud'})

test = pd.read_csv('/data/examples/may_the_4_be_with_u/show_me_the_money/test.csv',
                   names=['id','step','type','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest'],
                   encoding = "utf-8", header=None)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,3,5,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,3,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [122]:
train.shape

(3817573, 11)

In [123]:
train.isnull().values.any()

False

In [124]:
train.head()

,id,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud
0,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,Fraud
1,0,323,PAYMENT,9092.69,C749189758,0.0,0.0,M1939669337,0.0,0.0,0.0
2,1,164,CASH_IN,23468.75,C525009948,0.0,23468.75,C65353523,77300.17,53831.42,0.0
3,2,228,CASH_OUT,39688.11,C1311063915,0.0,0.0,C1581032686,168913.26,208601.37,0.0
4,3,229,CASH_OUT,179016.96,C700594269,103485.0,0.0,C90149178,2483034.47,2662051.42,0.0


In [125]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817573 entries, 0 to 3817572
Data columns (total 11 columns):
id                object
step              object
type              object
amount            object
nameOrig          object
oldBalanceOrig    object
newBalanceOrig    object
nameDest          object
oldBalanceDest    object
newBalanceDest    object
isFraud           object
dtypes: object(11)
memory usage: 320.4+ MB


In [126]:
test.shape

(2545048, 10)

In [127]:
test.isnull().values.any()

False

In [128]:
test.head()

,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
0,id,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
1,1,329,PAYMENT,41909.68,C1965438439,18391.0,0.0,M296912999,0.0,0.0
2,2,205,CASH_IN,57645.11,C2061644565,3162376.19,3220021.31,C2046839454,701205.57,643617.16
3,3,299,PAYMENT,41911.35,C630506216,9218.0,0.0,M1895361660,0.0,0.0
4,4,165,PAYMENT,10297.71,C1059151844,29739.0,19441.29,M830221248,0.0,0.0


### 匯出
train.to_csv('show_me_the_money_train.csv',index=False)

test.to_csv('show_me_the_money_test.csv',index=False)

In [129]:
print('\n The types of fraudulent transactions are {}'.format(\
list(train.loc[train.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' 
                                                             # & 'TRANSFER'

dfFraudTransfer = train.loc[(train.isFraud == 1) & (train.type == 'TRANSFER')]
dfFraudCashout = train.loc[(train.isFraud == 1) & (train.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) # 4097

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116


 The types of fraudulent transactions are ['CASH_OUT', 'TRANSFER']

 The number of fraudulent TRANSFERs = 2421

 The number of fraudulent CASH_OUTs = 2426


In [130]:
print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(train.loc[train.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False


Are there any merchants among originator accounts for CASH_IN transactions? False


In [131]:
print('\nAre there any merchants among destination accounts for CASH_OUT \
transactions? {}'.format(\
(train.loc[train.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False


Are there any merchants among destination accounts for CASH_OUT transactions? False


In [132]:
print('\nAre there merchants among any originator accounts? {}'.format(\
      train.nameOrig.str.contains('M').any())) # False

print('\nAre there any transactions having merchants among destination accounts\
 other than the PAYMENT type? {}'.format(\
(train.loc[train.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False


Are there merchants among any originator accounts? False

Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [133]:
print('\nWithin fraudulent transactions, are there destinations for TRANSFERS \
that are also originators for CASH_OUTs? {}'.format(\
(dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False
dfNotFraud = train.loc[train.isFraud == 0]


Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


In [134]:
print('\nFraudulent TRANSFERs whose destination accounts are originators of \
genuine CASH_OUTs: \n\n{}'.format(dfFraudTransfer.loc[dfFraudTransfer.nameDest.\
isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]))


Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

              id step      type  amount     nameOrig oldBalanceOrig  \
1139486  1139485  738  TRANSFER  814690  C2029041842         814690   

        newBalanceOrig     nameDest oldBalanceDest newBalanceDest isFraud  
1139486              0  C1023330867              0              0       1  


In [135]:
print('\nFraudulent TRANSFER to C423543548 occured at step = 486 whereas \
genuine CASH_OUT from this account occured earlier at step = {}'.format(\
dfNotFraud.loc[(dfNotFraud.type == 'CASH_OUT') & (dfNotFraud.nameOrig == \
                           'C1023330867')].step.values)) # 185


Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = [132]


In [136]:
X = train.loc[(train.type == 'TRANSFER') | (train.type == 'CASH_OUT')]

randomState = 5
np.random.seed(randomState)

#X = X.loc[np.random.choice(X.index, 100000, replace = False)]

Y = X['isFraud']
del X['isFraud']

# Eliminate columns shown to be irrelevant for analysis in the EDA
X = X.drop(['id', 'nameOrig', 'nameDest'], axis = 1)

# Binary-encoding of labelled data in 'type'
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int) # convert dtype('O') to dtype(int)

[^1]: [Predicting Fraud in Financial Payment Services | Kaggle](https://www.kaggle.com/arjunjoshua/predicting-fraud-in-financial-payment-services)

In [137]:
X.head()

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest
3,228,1,39688.11,0.0,0.0,168913.26,208601.37
4,229,1,179016.96,103485.0,0.0,2483034.47,2662051.42
5,709,1,77155.99,143307.0,66151.01,21213865.83,21291021.82
6,205,1,542294.41,0.0,0.0,3875179.23,4417473.64
9,154,1,3106.77,0.0,0.0,1002812.92,914313.38


In [138]:
Y.head()

3    0.0
4    0.0
5    0.0
6    0.0
9    0.0
Name: isFraud, dtype: object

In [139]:
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]
print('\nThe fraction of fraudulent transactions with \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceDest == 0) & \
(Xfraud.newBalanceDest == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud))))

print('\nThe fraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceDest == 0) & \
(XnonFraud.newBalanceDest == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud))))


The fraction of fraudulent transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.4961832061068702

The fraction of genuine transactions with 'oldBalanceDest' = newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0005850825439118876


In [140]:
X.loc[(X.oldBalanceDest == 0) & (X.newBalanceDest == 0) & (X.amount != 0), \
      ['oldBalanceDest', 'newBalanceDest']] = - 1

In [141]:
X.loc[(X.oldBalanceOrig == 0) & (X.newBalanceOrig == 0) & (X.amount != 0), \
      ['oldBalanceOrig', 'newBalanceOrig']] = np.nan

In [142]:
X['errorBalanceOrig'] = X.newBalanceOrig + X.amount - X.oldBalanceOrig
X['errorBalanceDest'] = X.oldBalanceDest + X.amount - X.newBalanceDest

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [143]:
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz

In [195]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, \
                                                random_state = randomState)

In [194]:
Y=Y.as_matrix().astype('float32')

In [162]:
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(1329834, 7)
(332459, 7)
(1329834,)
(332459,)


In [163]:
X_train = X_train.dropna(axis=0, how='any')
X_valid = X_valid.dropna(axis=0, how='any')
y_train = y_train.dropna(axis=0, how='any')
y_valid = y_valid.dropna(axis=0, how='any')

In [164]:
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(712465, 7)
(178189, 7)
(1329834,)
(332459,)


In [167]:
y_train = y_train.as_matrix()
y_valid = y_valid.as_matrix()

In [199]:
X_train = X_train.as_matrix().astype('float32')
X_valid = X_valid.as_matrix().astype('float32')

In [198]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332459 entries, 3335535 to 74393
Data columns (total 7 columns):
step              332459 non-null object
type              332459 non-null int64
amount            332459 non-null object
oldBalanceOrig    178189 non-null object
newBalanceOrig    178189 non-null object
oldBalanceDest    332459 non-null object
newBalanceDest    332459 non-null object
dtypes: int64(1), object(6)
memory usage: 20.3+ MB


In [201]:
X_train.shape,y_train.shape

((1329834, 7), (1329834,))

In [200]:
import xgboost

eval_set =  [(X_valid, y_valid)]
# parameters={'learning_rate':[0.05,0.1,0.2]}
model = xgboost.XGBClassifier()
# best_model = sklearn.model_selection.GridSearchCV(model, parameters,cv=2)
# best_model.fit(x_train, y_train,early_stopping_rounds=10, eval_metric="auc",
#           eval_set=eval_set, verbose=False)
model.fit(X_train, y_train,early_stopping_rounds=10)
# , eval_metric="auc",
#           eval_set=eval_set, verbose=True)

IndexError: list index out of range

In [ ]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['Fraud'] = 您的預測結果
submission.to_csv('您的檔名.csv',index=False)